In [1]:
import torch
from torch.utils.data import DataLoader
from torch import nn

import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

mnist_train = datasets.MNIST(root='./data', download=True, train=True, transform=ToTensor())
mnist_test  = datasets.MNIST(root='./data', download=True, train=False, transform=ToTensor())

train_dataloader = DataLoader(mnist_train, batch_size=32, shuffle=True)
test_dataloader  = DataLoader(mnist_test,  batch_size=32, shuffle=True)

model = nn.Sequential(
    nn.Linear(784, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
)

loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for i in range(0, 10):
    loss_sum = 0
    for X, y in train_dataloader:
        X = X.reshape((-1, 784))
        y = (y == 0).type(torch.float32).reshape((-1, 1))

        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    print(loss_sum)

# Evaluate on the test set:
# - set model.eval() and use torch.no_grad()
# - iterate over test_dataloader
# - reshape X to (-1, 784)
# - convert labels to binary: (y == 0).reshape((-1, 1))  (keep as bool for comparison)
# - compute outputs = sigmoid(model(X))
# - compute correct predictions: (outputs > 0.5) == y
# - accumulate total and correct counts and print accuracy (correct / total)
model.eval()
with torch.no_grad():
    accurate = 0
    total = 0
    for X, y in test_dataloader:
        X = X.reshape((-1, 784))
        y = (y == 0).reshape((-1, 1))

        outputs = nn.functional.sigmoid(model(X))
        correct_pred = ((outputs > 0.5) == y)

        total += correct_pred.size(0)
        accurate += correct_pred.type(torch.int).sum().item()

    print(accurate / total)

72.09383676154539
34.66346067969425
20.690170244460205
15.49028006494791
10.357178252879294
9.219604282158116
7.161865293334245
5.661256062815255
4.134548687723694
3.3387377083764536
0.9978
